In [96]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/Victoria/anaconda3/lib/python3.7/site-packages (0.0)


In [97]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [98]:
import pandas as pd

In [99]:
#Read CSV & do basic cleaning
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [100]:
df=df.drop(['koi_period_err1', 'koi_time0bk_err1', 'koi_impact_err2','koi_duration_err1','koi_depth_err1','koi_prad_err2','koi_insol_err2','koi_steff_err2','koi_srad_err2','dec'], axis=1)

In [101]:
# Set features. This will also be used as your x values.
#selected_features = df[['koi_pdisposition', 'koi_score', 'koi fpﬂag nt', 'koi fpﬂag ss', 'koi depth']]
X=df.drop('koi_disposition', axis=1)

In [102]:
y=df['koi_disposition']

In [103]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, random_state=43 )

In [104]:
#Pre-processing

# Scale your data
#from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [105]:
sc = MinMaxScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [106]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc_model=RandomForestClassifier(n_estimators=200)
rfc_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [107]:
model_params = {
    'n_estimators': [50, 100, 200, 250],
    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
    'min_samples_split': [2, 4, 6]
}

In [108]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(rfc_model, model_params, cv=5)
tuned_model = clf.fit(X, y)

In [109]:
pred_rfc=rfc_model.predict(X_test)

In [92]:
print(tuned_model.best_estimator_.get_params())

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 0.25, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 4, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [93]:
print(f"Training Data Score: {tuned_model.score(X_train, y_train)}")
print(f"Testing Data Score: {tuned_model.score(X_test, y_test)}")

Training Data Score: 0.5793991416309013
Testing Data Score: 0.6061472480343102


In [94]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))

                precision    recall  f1-score   support

     CANDIDATE       0.81      0.74      0.77       338
     CONFIRMED       0.77      0.81      0.79       333
FALSE POSITIVE       0.98      1.00      0.99       728

      accuracy                           0.89      1399
     macro avg       0.85      0.85      0.85      1399
  weighted avg       0.89      0.89      0.89      1399

[[249  79  10]
 [ 55 270   8]
 [  2   0 726]]


In [95]:
print(f"Training Data Score: {rfc_model.score(X_train, y_train)}")
print(f"Testing Data Score: {rfc_model.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8899213724088635


In [110]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'victoria2.sav'
joblib.dump(rfc_model, filename)

['victoria2.sav']